# Actor Screen Time by Credits Order
Does the order in which actors appear in the credits correspond to their screentime?

In [59]:
import tempfile
import unidecode
import subprocess as sp
from rekall.video_interval_collection import VideoIntervalCollection
import os

In [7]:
# Download stars.json
tmp_path = tempfile.NamedTemporaryFile(suffix='.json').name
stars_file = "gs://esper/movie-metadata/stars.json"
result = sp.check_call('gsutil cp {} {}'.format(stars_file, tmp_path), shell=True)

In [96]:
def decode_string(s):
    s = unidecode.unidecode(s)
    s = s.replace('ô', 'o')
    return s

with open(tmp_path, 'r') as f:
    data = json.load(f)
    for video_path in data:
        data[video_path] = [unidecode.unidecode(s)
                            for s in data[video_path] if len(s) > 0]

In [95]:
# get ranked order of main characters by screen time
# load all identities
fca_qs = FaceCharacterActor.objects.filter(
    face__frame__regularly_sampled=True,
    face__frame__video__ignore_film=False
).exclude(
    face__frame__video__genres__name="animation"
).annotate(
    min_frame=F('face__frame__number'),
    max_frame=F('face__frame__number'),
    actor_name=F('characteractor__actor__name'),
    video_id=F('characteractor__video_id')
).all()

faces_with_identity = VideoIntervalCollection.from_django_qs(
    fca_qs,
    with_payload=lambda row: unidecode.unidecode(row.actor_name)
)

In [98]:
def fold_fn(acc, intrvl):
    if intrvl.payload in acc:
        acc[intrvl.payload] += 1
    else:
        acc[intrvl.payload] = 1
    return acc
actor_counts = {}
for video_id in faces_with_identity.get_allintervals().keys():
    actor_counts[video_id] = faces_with_identity.get_intervallist(video_id).fold(fold_fn, {})

In [99]:
ordered_actor_counts = {
    video_id: sorted([(k, v) for k, v in actor_counts[video_id].items()], key=lambda kv: -1 * kv[1])
    for video_id in actor_counts
}

In [100]:
def videos_with_exact_same_order():
    total_videos = len(ordered_actor_counts.keys())
    same_order = 0
    for video_id in ordered_actor_counts:
        video = Video.objects.get(id=video_id)
        path = video.path
        credits_list = data[os.path.basename(path).split('.')[0]]
        screen_time_list = [actor_name for actor_name, _ in ordered_actor_counts[video_id]]
        if credits_list == screen_time_list:
            same_order += 1
    return same_order, total_videos

In [101]:
videos_with_exact_same_order()

(9, 90)

In [102]:
def videos_with_same_actor_in_position_x(x):
    total_videos = 0
    same_position_x = 0
    for video_id in ordered_actor_counts:
        video = Video.objects.get(id=video_id)
        path = video.path
        credits_list = data[os.path.basename(path).split('.')[0]]
        screen_time_list = [actor_name for actor_name, _ in ordered_actor_counts[video_id]]
        if x >= len(credits_list) or x >= len(screen_time_list):
            continue
        if credits_list[x] == screen_time_list[x]:
            same_position_x += 1
        total_videos += 1
    return same_position_x, total_videos

In [103]:
videos_with_same_actor_in_position_x(0)

(64, 90)

In [104]:
videos_with_same_actor_in_position_x(1)

(37, 89)

In [105]:
videos_with_same_actor_in_position_x(2)

(30, 80)

In [106]:
videos_with_same_actor_in_position_x(3)

(15, 73)

In [124]:
def get_actor_gender(actor_name):
    if (ActorGender.objects.filter(
            actor__name=actor_name
        ).annotate(gender_name=F('gender__name')).count() == 0):
        return 'U'
    return ActorGender.objects.filter(
        actor__name=actor_name
    ).annotate(gender_name=F('gender__name')).all()[0].gender_name
    
def get_actors_position_incorrect(x, gender_pairs=None):
    '''Get actors who appear in position x in the credits list but whose screen time
    does not put them at position x.'''
    actors_jumped_up = []
    actors_jumped_down = []
    for video_id in ordered_actor_counts:
        video = Video.objects.get(id=video_id)
        path = video.path
        credits_list = data[os.path.basename(path).split('.')[0]]
        screen_time_list = [actor_name for actor_name, _ in ordered_actor_counts[video_id]]
        if x >= len(credits_list) or x >= len(screen_time_list):
            continue
        if credits_list[x] == screen_time_list[x]:
            continue
        actor_at_credits_x = credits_list[x]
        if actor_at_credits_x not in screen_time_list:
            print(video.title, video.year, actor_at_credits_x, credits_list, ordered_actor_counts[video_id])
            continue
        screen_time_position = screen_time_list.index(actor_at_credits_x)
        credits_gender = get_actor_gender(actor_at_credits_x)
        screen_time_gender = get_actor_gender(screen_time_list[x])
        if gender_pairs is not None:
            if credits_gender != gender_pairs[0] or screen_time_gender != gender_pairs[1]:
                continue
        if screen_time_position > x:
            actors_jumped_up.append((
                video.title, video.year, actor_at_credits_x, credits_gender,
                screen_time_position, screen_time_list[x], screen_time_gender
            ))
        else:
            actors_jumped_down.append((
                video.title, video.year, actor_at_credits_x, credits_gender,
                screen_time_position, screen_time_list[x], screen_time_gender
            ))
    return actors_jumped_up, actors_jumped_down

In [138]:
get_actors_position_incorrect(0, ('M', 'F'))

Beneath the Planet of the Apes 1970 james franciscus ['james franciscus', 'kim hunter', 'maurice evans', 'linda harrison', 'charlton heston'] [('charlton heston', 2877), ('kim hunter', 821), ('linda harrison', 789), ('maurice evans', 426)]


([('Star Wars: The Force Awakens',
   2015,
   'harrison ford',
   'M',
   2,
   'daisy ridley',
   'F'),
  ('Batman & Robin',
   1997,
   'arnold schwarzenegger',
   'M',
   5,
   'uma thurman',
   'F'),
  ('Notorious', 1946, 'cary grant', 'M', 1, 'ingrid bergman', 'F'),
  ('Rebecca', 1940, 'laurence olivier', 'M', 1, 'joan fontaine', 'F'),
  ('Alice in Wonderland', 2010, 'johnny depp', 'M', 1, 'mia wasikowska', 'F'),
  ('Star Wars: Episode V - The Empire Strikes Back',
   1980,
   'mark hamill',
   'M',
   1,
   'carrie fisher',
   'F'),
  ('The Color Purple', 1985, 'danny glover', 'M', 2, 'whoopi goldberg', 'F')],
 [])

In [136]:
get_actors_position_incorrect(1, ('M', 'F'))

The Wizard of Oz 1925 dorothy dwan ['larry semon', 'dorothy dwan', 'oliver hardy', 'spencer bell', 'bryant washburn', 'virginia pearson', 'charles murray'] [('larry semon', 495), ('oliver hardy', 467), ('josef swickard', 310), ('bryant washburn', 291), ('virginia pearson', 227), ('charles murray', 164)]


([('Birdman or (The Unexpected Virtue of Ignorance)',
   2014,
   'zach galifianakis',
   'M',
   3,
   'emma stone',
   'F'),
  ('Captain Fantastic', 2016, 'frank langella', 'M', 3, 'kathryn hahn', 'F'),
  ('Brazil', 1985, 'robert de niro', 'M', 6, 'kim greist', 'F'),
  ('GoldenEye', 1995, 'sean bean', 'M', 2, 'izabella scorupco', 'F'),
  ('Harry Potter and the Deathly Hallows: Part 1',
   2010,
   'rupert grint',
   'M',
   2,
   'emma watson',
   'F'),
  ('Harry Potter and the Deathly Hallows: Part 2',
   2011,
   'rupert grint',
   'M',
   2,
   'emma watson',
   'F'),
  ('Harry Potter and the Goblet of Fire',
   2005,
   'rupert grint',
   'M',
   2,
   'emma watson',
   'F'),
  ('Harry Potter and the Order of the Phoenix',
   2007,
   'rupert grint',
   'M',
   2,
   'emma watson',
   'F'),
  ('Star Wars: Episode I - The Phantom Menace',
   1999,
   'ewan mcgregor',
   'M',
   3,
   'natalie portman',
   'F'),
  ('Bessie',
   2015,
   'michael kenneth williams',
   'M',
   2,
   

In [122]:
get_actors_position_incorrect(2)

A Clockwork Orange 1971 adrienne corri ['malcolm mcdowell', 'patrick magee', 'adrienne corri', 'miriam karlin'] [('malcolm mcdowell', 3657), ('patrick magee', 998), ('anthony sharp', 584), ('warren clarke', 546), ('godfrey quigley', 545), ('michael bates', 513), ('james marcus', 388), ('philip stone', 378), ('carl duering', 223), ('david prowse', 216), ('sheila raynor', 196), ('madge ryan', 174), ('pauline taylor', 119), ('miriam karlin', 75)]
Back to the Future 1985 lea thompson ['michael j. fox', 'christopher lloyd', 'lea thompson', 'crispin glover'] [('michael j. fox', 3928), ('christopher lloyd', 1130), ('crispin glover', 997), ('frances lee mccain', 924)]
Erin Brockovich 2000 aaron eckhart ['julia roberts', 'albert finney', 'aaron eckhart'] [('julia roberts', 4879), ('albert finney', 2053), ('ron altomare', 854)]


([('12 Years a Slave',
   2013,
   'benedict cumberbatch',
   'M',
   5,
   'paul giamatti',
   'M'),
  ('Star Wars: The Force Awakens',
   2015,
   'carrie fisher',
   'F',
   5,
   'harrison ford',
   'M'),
  ('All About Eve', 1950, 'george sanders', 'M', 3, 'celeste holm', 'F'),
  ('Apollo 13', 1995, 'bill paxton', 'M', 5, 'kathleen quinlan', 'F'),
  ('2012', 2009, 'amanda peet', 'F', 4, 'oliver platt', 'M'),
  ('Beneath the Planet of the Apes',
   1970,
   'maurice evans',
   'M',
   3,
   'linda harrison',
   'F'),
  ('Die Hard', 1988, 'alexander godunov', 'M', 4, 'bonnie bedelia', 'F'),
  ('The Lord of the Rings: The Fellowship of the Ring',
   2001,
   'liv tyler',
   'F',
   13,
   'ian mckellen',
   'M'),
  ('The Lord of the Rings: The Two Towers',
   2002,
   'liv tyler',
   'F',
   13,
   'sean astin',
   'M'),
  ('Yojimbo', 1961, 'yoko tsukasa', 'U', 6, 'atsushi watanabe', 'M'),
  ('Carol', 2015, 'sarah paulson', 'F', 3, 'kyle chandler', 'M'),
  ("One Flew Over the Cuckoo's

In [123]:
get_actors_position_incorrect(3)

Battle for the Planet of the Apes 1973 severn darden ['roddy mcdowall', 'claude akins', 'natalie trundy', 'severn darden', 'lew ayres', 'paul williams', 'john huston'] [('austin stoker', 960), ('roddy mcdowall', 751), ('natalie trundy', 630), ('paul williams', 613), ('claude akins', 350)]
Dances with Wolves 1990 rodney grant ['kevin costner', 'mary mcdonnell', 'graham greene', 'rodney grant'] [('kevin costner', 6132), ('graham greene', 1654), ('mary mcdonnell', 1582), ('rodney a. grant', 975), ("floyd 'red crow' westerman", 622), ('nathan lee chasing his horse', 456), ('tantoo cardinal', 410), ('jimmy herman', 292), ('maury chaykin', 202), ('robert pastorelli', 180)]
Big Eyes 2014 jon polito ['amy adams', 'christoph waltz', 'danny huston', 'jon polito', 'krysten ritter', 'jason schwartzman', 'terence stamp'] [('christoph waltz', 2682), ('amy adams', 2637), ('danny huston', 1432), ('delaney raye', 605), ('james saito', 312)]
Brooklyn 2015 jim broadbent ['saoirse ronan', 'domhnall gleeso

([('12 Years a Slave', 2013, 'paul dano', 'M', 7, "lupita nyong'o", 'F'),
  ('A Clockwork Orange', 1971, 'miriam karlin', 'F', 13, 'warren clarke', 'M'),
  ('Star Wars: The Force Awakens',
   2015,
   'adam driver',
   'M',
   4,
   'oscar isaac',
   'M'),
  ('Airplane II: The Sequel',
   1982,
   'chad everett',
   'M',
   5,
   'peter graves',
   'M'),
  ('Apollo 13', 1995, 'gary sinise', 'M', 4, 'kevin bacon', 'M'),
  ('Birdman or (The Unexpected Virtue of Ignorance)',
   2014,
   'andrea riseborough',
   'F',
   4,
   'zach galifianakis',
   'M'),
  ('Braveheart', 1995, 'catherine mccormack', 'F', 4, 'james robinson', 'M'),
  ('Casino Royale', 2006, 'giancarlo giannini', 'M', 4, 'judi dench', 'F'),
  ('Yojimbo', 1961, 'isuzu yamada', 'F', 5, 'tatsuya nakadai', 'M'),
  ('Jaws', 1975, 'lorraine gary', 'F', 4, 'murray hamilton', 'M'),
  ('Carol', 2015, 'jake lacy', 'M', 4, 'sarah paulson', 'F'),
  ('Harry Potter and the Chamber of Secrets',
   2002,
   'kenneth branagh',
   'M',
   4,